In [ ]:
results_directory="/private/home/marialomeli/LASER/tasks/bucc/embed"
# load candidate.tsv files and compute precision and recall to plot the P/R curve
# can just run bucc.py with the given file name and collect the output
# we don't need to run for all values of k, we can reuse one of the files 
# with lots of neighbours and select a subset 